<center>
    <h1> Predicción de calidad del vino y explicación con LIME </h2>
</center>

## Contenidos

* [Introducción](#Introducción)
* [Bibliotecas](#Bibliotecas)
* [Datos](#Datos)
* [Modelo Caja Negra](#Modelo-Caja-Negra)
* [LimeTabularExplainer](#LimeTabularExplainer)
* [Ejercicio](#Ejercicio)

## Introducción

En el siguiente notebook, se ejemplifica el uso de [LIME](https://dl.acm.org/doi/abs/10.1145/2939672.2939778) para explicar localmente un modelo de red neuronal que predice la calidad de un vino.

# Bibliotecas

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install lime

In [5]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lime import lime_tabular, submodular_pick
from lime import submodular_pick
from keras.layers import Dense
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

## Datos

El conjunto de datos se puede descargar desde el siguiente enlace: [Red Wine Quality](https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009).

In [ ]:
!gdown https://drive.google.com/uc?id=1HXo_Pluc28lWf3XK_IdOdnKHIBeuxDaQ

In [ ]:
df = pd.read_csv("winequality-red.csv")
df.head()

In [ ]:
X = df.iloc[:,0:11]
y = np.array(df.quality)

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X_train, y_train,test_size=0.5,random_state = 42)

X_train.shape, X_val.shape, X_test.shape

## Modelo Caja Negra

El modelo de caja negra a explicar es una  *Feedforward Neural Network* (FNNN).

### Arquitectura

La FNN en cuestión está basada en el siguiente [ejemplo](https://www.analyticsvidhya.com/blog/2021/07/plunging-into-deep-learning-carrying-a-red-wine/).

In [ ]:
model = Sequential([
    Dense(512, activation='relu', input_shape=[11]),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dense(1),
])

In [ ]:
model.summary()

### Entrenamiento

In [12]:
model.compile(
    optimizer="adam",
    loss="mae",
)

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    batch_size=256,
    epochs=200,
    verbose=0,
)

In [ ]:
plt.figure()
plt.plot(history.history['loss'], label="Train")
plt.plot(history.history['val_loss'], label="Validation")
plt.xlabel("Época")
plt.ylabel("Error")
plt.legend(loc="best")
plt.show()

### Evaluación

In [ ]:
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

In [ ]:
mae, mse

## LimeTabularExplainer

In [17]:
explainer = lime_tabular.LimeTabularExplainer(
    X_train.values,
    feature_names=X_train.columns,
    class_names=["quality"],
    verbose=True,
    mode="regression",
)

A continuación, explicaremos localmente una predicción para una instancia del conjunto de prueba usando LIME.

In [18]:
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
index = 0
instance = X_test.iloc[index]
instance

In [ ]:
local_explanation = explainer.explain_instance(instance, model.predict, num_samples=100000)

In [ ]:
local_explanation

In [ ]:
local_explanation.as_list()

In [ ]:
local_explanation.show_in_notebook(show_table=True, show_all=False)

In [ ]:
local_explanation.as_pyplot_figure(label=1)
plt.show()

## Pregunta

¿Qué puede interpretar de los resultados que entrega LIME?



## Ejercicio

Usar el método ``SubmodularPick`` para generar una explicación global asociada al método SP-Lime. El siguiente [artículo](https://medium.com/@ML-STATS/lime-local-interpretable-model-agnostic-explanations-part-3-12bb30fc2a03) puede ser de ayuda.